In [50]:
import pandas as pd

# Dados Gerais
dg_bairros = pd.read_csv("database_csv/bairro.csv")
dg_sub_bacias = pd.read_csv("database_csv/sub_bacias.csv")
dg_grid = pd.read_csv("database_csv/h3_grid_res8.csv")

# # Dados de COR, 1746, e pontos supervisionados
cor_ocorrencias = pd.read_csv("database_csv/ocorrencias.csv")
cor_ocorrencias_pop_filtrado = pd.read_csv("database_csv/ocorrencias_pop_filtrado.csv")
cor_pop = pd.read_csv("database_csv/procedimento_operacional_padrao.csv")
datario_chamado_1746_filtrado = pd.read_csv("database_csv/chamado_2.csv")
ra_psalagamento = pd.read_csv("database_csv/ponto_supervisionado_alagamento.csv")

# # Dados Pluviométricos
cor_est_alerta_rio = pd.read_csv("database_csv/estacoes_alertario.csv")
cor_est_inea = pd.read_csv("database_csv/estacoes_inea.csv")
cor_est_cemaden = pd.read_csv("database_csv/estacoes_cemaden.csv")
cor_est_websirene = pd.read_csv("database_csv/estacoes_websirene.csv")

cor_tx_precip_websirene = pd.read_csv("database_csv/taxa_precipitacao_websirene.csv")
cor_tx_precip_alerta_rio = pd.read_csv("database_csv/taxa_precipitacao_alertario_5min.csv")
cor_tx_precip_inea = pd.read_csv("database_csv/taxa_precipitacao_inea.csv")
cor_tx_precip_cemaden = pd.read_csv("database_csv/taxa_precipitacao_cemaden.csv")

# # Dados Fluviométricos e dados de reservtórios
ra_lam_inea = pd.read_csv("database_csv/lamina_agua_inea.csv")
ra_lam_via = pd.read_csv("database_csv/nivel_lamina_agua_via.csv")
ra_nivel_reservatorio = pd.read_csv("database_csv/nivel_reservatorio.csv")

# # Radares_metereológicos
# cor_tx_precip_guaratiba = pd.read_csv("database_csv/taxa_precipitacao_guaratiba.csv")

/tmp/ipykernel_96705/1182996995.py:12: DtypeWarning: Columns (21,22,29) have mixed types. Specify dtype option on import or set low_memory=False.
  datario_chamado_1746_filtrado = pd.read_csv("database_csv/chamado_2.csv")


In [51]:
# Como criar um ranking de chuvas?

# Filtrando o dataset para os dias nos quais houveram chuvas
cor_tx_precip_alerta_rio = cor_tx_precip_alerta_rio[(cor_tx_precip_alerta_rio["acumulado_chuva_24h"]>0)]

# Ordenando os dados por data
cor_tx_precip_alerta_rio = cor_tx_precip_alerta_rio.sort_values(by='data_medicao', ascending=False)

# # Transformando os dados em mm/h

colunas_acumulado_chuva = ['acumulado_chuva_5min', 'acumulado_chuva_10min', 'acumulado_chuva_15min',
                           'acumulado_chuva_30min', 'acumulado_chuva_1h', 'acumulado_chuva_2h',
                           'acumulado_chuva_3h', 'acumulado_chuva_4h', 'acumulado_chuva_6h',
                           'acumulado_chuva_12h', 'acumulado_chuva_24h']

# Converter os acumulados de chuva para mm/h
for coluna in colunas_acumulado_chuva:
    # Extrair o período de tempo da coluna
    tempo_str = coluna.split('_')[2]
    if tempo_str[-3:] == "min":
        tempo = int(tempo_str[:-3]) / 60
    else:
        tempo = int(tempo_str[:-1])
    
    # Converter para mm/h
    cor_tx_precip_alerta_rio[coluna + '_mmh'] = cor_tx_precip_alerta_rio[coluna] / tempo


cor_tx_precip_alerta_rio['data_medicao'] = pd.to_datetime(cor_tx_precip_alerta_rio['data_medicao'])
dias_unicos = pd.unique(cor_tx_precip_alerta_rio['data_medicao'].dt.date)

In [60]:
# transformando para data
# Adicionando a Data
cor_tx_precip_websirene["data_medicao"] = cor_tx_precip_websirene['primary_key'].apply(lambda x: x.split('_')[1])
# cor_tx_precip_websirene["hora_medicao"] = cor_tx_precip_websirene['primary_key'].apply(lambda x: x.split()[1] if len(x.split()) >= 2 else None)
cor_tx_precip_websirene["data_medicao"] =  pd.to_datetime(cor_tx_precip_websirene["data_medicao"])
# cor_tx_precip_websirene["hora_medicao"] =  pd.to_datetime(cor_tx_precip_websirene["hora_medicao"])

# Ordenando por valor
cor_tx_precip_websirene = cor_tx_precip_websirene.sort_values(by='data_medicao', ascending=False)

# Lista das colunas que precisam ser corrigidas
colunas_a_corrigir = ['acumulado_chuva_15_min', 'acumulado_chuva_1_h', 'acumulado_chuva_4_h', 'acumulado_chuva_24_h', 'acumulado_chuva_96_h']

# Substituir NaN por 0 nas colunas especificadas
cor_tx_precip_websirene[colunas_a_corrigir] = cor_tx_precip_websirene[colunas_a_corrigir].fillna(0)

# cor_tx_precip_websirene = cor_tx_precip_websirene[cor_tx_precip_websirene["id_estacao"] == 5]

# Evento chuvoso
def criar_evento_chuvoso(df):
    eventos = {}
    evento_num = 0
    is_evento = False
    for index, row in df.iterrows():
        id_estacao = row['id_estacao']
        acumulado_4h = row['acumulado_chuva_4_h']
        if acumulado_4h == 0:
            is_evento = False
        else:
            if not is_evento:
                evento_num += 1
            evento_atual = f"{id_estacao}_evento_{evento_num}"
            is_evento = True
            eventos[index] = evento_atual
    return eventos

# Criar a coluna 'evento_chuvoso' usando a função criar_evento_chuvoso
eventos = criar_evento_chuvoso(cor_tx_precip_websirene)
cor_tx_precip_websirene['evento_chuvoso'] = pd.Series(eventos)

# Preencher os valores faltantes com 0
cor_tx_precip_websirene['evento_chuvoso'].fillna(0, inplace=True)

# Exibir o DataFrame resultante
cor_tx_precip_websirene[cor_tx_precip_websirene["evento_chuvoso"] == "5_evento_1"]



,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,horario,data_particao,data_medicao,evento_chuvoso
1453568,5_2024-03-11 23:50:00,5,0.0,0.0,0.6,0.6,14.0,23:50:00,2024-03-11,2024-03-11 23:50:00,5_evento_1
661197,5_2024-03-11 23:40:00,5,0.0,0.0,0.6,0.6,14.0,23:40:00,2024-03-11,2024-03-11 23:40:00,5_evento_1
1156479,5_2024-03-11 23:25:00,5,0.0,0.0,0.6,0.6,14.0,23:25:00,2024-03-11,2024-03-11 23:25:00,5_evento_1
362647,5_2024-03-11 23:15:00,5,0.0,0.0,0.6,0.6,14.0,23:15:00,2024-03-11,2024-03-11 23:15:00,5_evento_1
1800318,5_2024-03-11 22:55:00,5,0.0,0.0,0.6,0.6,14.0,22:55:00,2024-03-11,2024-03-11 22:55:00,5_evento_1
760143,5_2024-03-11 22:45:00,5,0.0,0.2,0.6,0.6,14.0,22:45:00,2024-03-11,2024-03-11 22:45:00,5_evento_1
1602138,5_2024-03-11 22:25:00,5,0.0,0.2,0.6,0.6,14.0,22:25:00,2024-03-11,2024-03-11 22:25:00,5_evento_1
1007604,5_2024-03-11 22:15:00,5,0.0,0.2,0.6,0.6,14.0,22:15:00,2024-03-11,2024-03-11 22:15:00,5_evento_1
461666,5_2024-03-11 21:50:00,5,0.0,0.4,0.4,0.4,13.8,21:50:00,2024-03-11,2024-03-11 21:50:00,5_evento_1
511742,5_2024-03-11 21:45:00,5,0.0,0.4,0.4,0.4,13.8,21:45:00,2024-03-11,2024-03-11 21:45:00,5_evento_1
